In [1]:
'''
删除S645中在S4169里出现过的数据
'''
import pandas as pd
import numpy as np

data_name = 'AB_645'

data = pd.read_csv('./AB-Bind-Database/{}.csv'.format(data_name)).values.tolist()
base_data = pd.read_csv('./S4169.csv').values.tolist()


base_dict = dict()
base_dict_reverse = dict()
for item in base_data:
    pdb_name = item[0]
    mutChain_mutation = item[4]
    ddg_abs = np.abs(float(item[5]))

    mutChain, mutation = mutChain_mutation.split(':')
    src_aa = mutation[0]
    pos = mutation[1:-1]
    tgt_aa = mutation[-1]

    key = '{}:{}_{}'.format(pdb_name, src_aa, tgt_aa)
    base_dict[key] = ddg_abs

    key = '{}:{}_{}'.format(pdb_name, tgt_aa, src_aa)
    base_dict_reverse[key] = ddg_abs



data_fix = list()
for item in data:
    pdb_name = item[0]
    mutChain_mutation = item[4]
    ddg_abs = np.abs(float(item[5]))
    mut_chain, mutation = mutChain_mutation.split(':')
    src_aa = mutation[0]
    pos = mutation[1:-1]
    tgt_aa = mutation[-1]

    key = '{}:{}_{}'.format(pdb_name, src_aa, tgt_aa)
    if key in base_dict.keys() and np.abs(base_dict[key] - ddg_abs) < 0.05:
        data_fix.append(item + [0])
    elif key in base_dict_reverse.keys() and np.abs(base_dict_reverse[key] - ddg_abs) < 0.05:
        data_fix.append(item + [0])
    else:
        data_fix.append(item + [1])

df = pd.DataFrame(data_fix)
df.columns = list(pd.read_csv('./AB-Bind-Database/{}.csv'.format(data_name)).columns) + ['not_redundant']
df.to_csv('./{}_delRedudant.csv'.format(data_name), index=False)
